# LASSO Regression

LASSO全称为The Least Absolute Shrinkage and Selection Operator, 是一种特征选择的方法，经过LASSO处理之后，就能知道训练样本的所有特征中哪些事有效特征。应为杜宇无效特征，其对应的权重为0。

LASSO的实际上是一种[线性回归](linear_regression.ipynb), 它在线性回归目标函数的基础上添加一个约束$\displaystyle\sum_{i = 1}^n|w_i| \le t$(也可以认为是在其目标函数上添加了$L_1$[正则](regularization.ipynb)，这是等价的)。这样在优化目标函数的时候，会有许多$w_i$取值为0。在添加约束之后，其优化的目标函数变成
$$
\underset{W, b}{arg\ min} \displaystyle\sum_{i = 1}^N (W^Tx_i + b -y_i)^2  + \lambda||W||_1\tag{1}
$$

观察目标函数可知，目标函数并不处处可导，那么其优化方式不能使用梯度下降。如果想要使用梯度下降法模型优化，可以使用如下的假设
$$
L_1(\theta) =\displaystyle\sum_{d = 1}^D\sqrt{\theta_d^2 + \varepsilon}
\tag{2}
$$
使用公式(2)代替公式(1)的右半部分后就可以使用梯度下降法、牛顿法等依靠梯度优化的方法

当然，优化LASSO的方法非常多，本笔记仅将详细的描述坐标下降法

## Coordinate Descent

坐标下降法每次针对一个属性进行更新，我们假设数据集有m个属性，n个样例，数据矩阵为$m \times n$ 的矩阵，那么目标函数为：
$$
W^* = \underset{W}{arg\ min} \displaystyle\sum_{i = 1}^n\bigg(y_i - \sum_{j= 1}^mx_{ij}W_j\bigg)^2 + 
\lambda\sum_{j=1}^m|W_j|\tag{3}
$$

假设现在沿着第$k$个维度做优化，对其求偏导有：
$$
\begin{aligned}
\frac{\partial L}{\partial W_k} 
=& \displaystyle\sum_{i = 1}^n-2x_{ik}\Big(y_i - \sum_{j = 1}^mx_{ij}W_j\Big) + 
\begin{cases}
\lambda & W_k > 0\\ 
[-\lambda, \lambda] & W_k = 0\\
-\lambda  &W_k < 0
\end{cases}\\
=& -2\displaystyle\sum_{i = 1}^nx_{ik}\Big(y_i - \sum_{j\not= k}^m x_{ij}W_j\Big) + 2W_k\sum_{i = 1}^nx_{ik}^2 + 
\begin{cases}
\lambda & W_k > 0\\ 
[-\lambda, \lambda] & W_k = 0\\
-\lambda  &W_k < 0
\end{cases}\\
\end{aligned}\tag{4}
$$


令
$$
\begin{aligned}
p_k =& -2\displaystyle\sum_{i = 1}^nx_{ik}\Big(y_i - \sum_{j\not= k}^m x_{ij}W_j\Big) \\
m_k =& 2W_k\sum_{i = 1}^nx_{ik}^2
\end{aligned}
$$

那么公式(4)可以写作
$$
\frac{\partial L}{\partial W_k}  = 
p_k + m_kW_k + 
\begin{cases}
\lambda & W_k > 0\\ 
[-\lambda, \lambda] & W_k = 0\\
-\lambda  &W_k < 0
\end{cases}
\tag{5}
$$

推导成这样的形式是因为这是对$W_k$的偏导。令偏导数为0得到极值，暂时先不考虑$W_k = 0$的情况，那么令偏导等于0就有
$$
p_k + m_kW^*_k + \lambda sign(W_k^*) = 0\tag{6}
$$
$$
W_k^* + \frac{1}{m_k}(p_k + \lambda sign(W_k^*)) = 0\tag{7}
$$
找出能满足方程的$w^*_k$即可找到极值点，这种情况只能分类讨论，首先明确$m_k  >0$

1. 当$p_k > \lambda$时：

   - 若$W_k^* > 0$时，$W_k^* + \frac{1}{m_k}(p_k + \lambda sign(W_k^*)) = 0$不可能，两个正数之和不可能为0；

   - 若$W_k^* < 0$时，$W_k^* + \frac{1}{m_k}(p_k + \lambda sign(W_k^*)) = 0$合理，因为一正一负；

2. 当$p_k < -\lambda$时：
	 - 若$W_k^* > 0$时，$W_k^* + \frac{1}{m_k}(p_k + \lambda sign(W_k^*)) = 0$ 合理，因为一正一负；
	 - 若$W_k^* < 0$时，$W_k^* + \frac{1}{m_k}(p_k + \lambda sign(W_k^*)) = 0$ 不可能，两个负数之和不可能为0;

3. 当$-\lambda \le p_k \le \lambda$时，无论$W_k^* > 0$或者$W_k^* < 0$均不可能成立，故考虑$W_k^* = 0$，

就这样得出了一个闭合解:
$$
W_k^* = 
\begin{cases}
-\frac{1}{m_k}(p_K - \lambda) & p_k > \lambda\\
-\frac{1}{m_k}(p_K + \lambda) & p_k < - \lambda\\
0
\end{cases}\tag{8}
$$

本模型的[示例](../code/6_lasso.ipynb)就是使用coordinate descent实现的。